In [1]:
import cv2
from keras.models import load_model
import numpy as np
import os

Using TensorFlow backend.


In [2]:
model = load_model('emojinator.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [3]:
def main():
    emojis = get_emojis()
    cap = cv2.VideoCapture(0)
    x, y, w, h = 300, 50, 350, 350

    while (cap.isOpened()):
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
        res = cv2.bitwise_and(img, img, mask=mask2)
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        median = cv2.GaussianBlur(gray, (5, 5), 0)

        kernel_square = np.ones((5, 5), np.uint8)
        dilation = cv2.dilate(median, kernel_square, iterations=2)
        opening = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_square)
        ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)

        thresh = thresh[y:y + h, x:x + w]
        contours,_ = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        if len(contours) > 0:
            contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(contour) > 2500:
                x, y, w1, h1 = cv2.boundingRect(contour)
                newImage = thresh[y:y + h1, x:x + w1]
                newImage = cv2.resize(newImage, (50, 50))
                pred_probab, pred_class = keras_predict(model, newImage)
                print(pred_class, pred_probab)
                img = overlay(img, emojis[pred_class], 400, 250, 90, 90)

        x, y, w, h = 300, 50, 350, 350
        cv2.imshow("Frame", img)
        cv2.imshow("Contours", thresh)
        k = cv2.waitKey(10)
        if k == 27:
            break

In [4]:

def keras_predict(model, image):
    processed = keras_process_image(image)
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class

In [5]:
def keras_process_image(img):
    image_x = 50
    image_y = 50
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

In [6]:
def get_emojis():
    emojis_folder = 'hand_emo/'
    emojis = []
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis


In [7]:
def overlay(image, emoji, x,y,w,h):
    emoji = cv2.resize(emoji, (w, h))
    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image

In [8]:
def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))


In [ ]:
keras_predict(model, np.zeros((50, 50, 1), dtype=np.uint8))
main()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
14 0.49963441
10 0.7771648
5 0.9953526
10 0.7169519
10 0.77831423
14 0.45899418
10 0.6717479
14 0.76721096
14 0.9080339
14 0.8598126
11 0.50763464
10 0.72346044
10 0.70702296
14 0.8873647
10 0.82774776
10 0.8139482
10 0.4728618
11 0.46182165
10 0.71316975
5 0.3295026
14 0.5015099
14 0.73376775
10 0.82019943
10 0.55514747
10 0.5170763
14 0.6683915
10 0.82121855
10 0.64963186
10 0.79506785
10 0.8484315
10 0.9113076
10 0.78561914
14 0.6529421
14 0.7385153
14 0.75999475
14 0.4855529
10 0.8356245
14 0.89453334
14 0.5470227
14 0.9594198
14 0.7410032
14 0.7068966
10 0.8358291
10 0.5301482
14 0.264224
14 0.5674374
10 0.91319543
10 0.5949179
10 0.8703967
10 0.73552626
10 0.88189507
10 0.86036783
14 0.8457817
14 0.6429177
14 0.53785247
14 0.6200549
10 0.5986525
14 0.7266603
11 0.82233876
10 0.93657404
14 0.86131704
10 0.5681494
10 0.80246395
14 0.74417806
14 0.8934496
10 0.5426257
14 0.8661002
10 0.9186904
10 0.8990564
5 0.77604663
10 0.70957524
10 0.86085755
1

5 0.9602628
5 0.97700715
5 0.9700281
5 0.9135524
5 0.97309124
5 0.32297844
11 0.9346669
5 0.95757055
5 0.7452484
5 0.9803223
5 0.9906489
5 0.7470573
5 0.9920838
5 0.96818113
5 0.39911473
5 0.9731073
5 0.9827664
3 0.34374207
5 0.98902345
3 0.4788542
5 0.96533525
5 0.9710938
5 0.9871171
5 0.8562319
5 0.8984119
5 0.915455
2 0.43738
2 0.89453715
5 0.33159631
2 0.34938347
11 0.3424001
2 0.61412513
2 0.55236834
11 0.562491
11 0.9490477
11 0.8660636
11 0.9602449
11 0.69789946
11 0.95867723
11 0.7386397
11 0.91021484
11 0.8562861
11 0.91948575
11 0.77189803
11 0.57975537
11 0.8660722
11 0.9290669
11 0.5083492
14 0.50293666
14 0.35982478
11 0.82999074
11 0.7082938
11 0.5753309
11 0.68142027
11 0.6682103
11 0.7499896
11 0.6158796
11 0.59832036
11 0.6196095
11 0.72021353
11 0.6905003
11 0.75759083
11 0.53087693
11 0.8084926
11 0.8446293
11 0.7190527
11 0.58975184
11 0.5581236
14 0.50515074
11 0.7196984
11 0.85333246
11 0.90957165
11 0.80427074
11 0.6280687
11 0.8433045
11 0.81487757
11 0.7379391


11 0.44672513
8 0.49149457
11 0.6023439
11 0.7331669
2 0.42172217
11 0.45421028
2 0.3051093
11 0.31461886
11 0.50269705
8 0.42935708
2 0.75710416
2 0.776682
2 0.76934355
11 0.3267858
11 0.4313798
11 0.6221567
11 0.5295962
11 0.5179045
11 0.45353958
2 0.42457712
11 0.3881179
11 0.3888941
11 0.7519035
11 0.51925635
2 0.4092789
11 0.81318295
2 0.33847433
11 0.5533879
11 0.51989245
2 0.3139948
11 0.3208259
10 0.40732262
11 0.45326722
11 0.4668515
11 0.5080969
11 0.400966
2 0.59688234
8 0.51696235
11 0.35982206
11 0.9145036
11 0.46338803
11 0.527273
11 0.30719194
11 0.77417743
11 0.34295225
11 0.47082955
5 0.9957241
8 0.66163605
11 0.84759897
2 0.2837045
2 0.38548753
11 0.76707363
11 0.7173102
11 0.6099377
11 0.48766905
2 0.56064826
2 0.51037407
11 0.40604505
2 0.45911366
2 0.558747
11 0.6190373
5 0.9660424
11 0.5561982
2 0.6852249
11 0.3692952
2 0.4852383
11 0.76840067
2 0.45685142
11 0.87495697
5 0.9972082
5 0.9979085
5 0.9981336
5 0.58147675
5 0.99788755
5 0.99739766
5 0.9980465
10 0.541

1 0.82521075
1 0.7793155
11 0.5805845
11 0.6850312
11 0.8870874
11 0.80310327
11 0.9367836
11 0.92904174
11 0.98478115
11 0.8846584
11 0.8690304
11 0.77612406
11 0.7677664
11 0.8020818
11 0.72957295
11 0.61423826
11 0.66806805
11 0.6620131
5 0.54724634
11 0.4548417
7 0.76305026
7 0.9581269
5 0.438112
11 0.54965764
5 0.60506254
5 0.6329727
5 0.82698685
5 0.805859
5 0.91808933
5 0.8607409
5 0.7514077
11 0.44262636
14 0.6315274
11 0.49256712
11 0.39995223
14 0.75265676
11 0.7414933
11 0.8422049
14 0.6501442
11 0.4538154
11 0.5899935
14 0.51985943
11 0.63709724
14 0.41942847
3 0.55744934
3 0.57659656
14 0.46373376
3 0.5860187
3 0.5783884
3 0.5185576
3 0.7104854
3 0.26779455
3 0.61640704
3 0.67642915
3 0.63761747
11 0.47491136
11 0.5835746
1 0.79975677
1 0.9248581
11 0.34616247
3 0.4072497
14 0.24136503
14 0.4588445
5 0.7231807
5 0.69376826
5 0.6317358
5 0.7004862
10 0.5106622
5 0.64621973
5 0.56058645
10 0.59346676
5 0.7198281
5 0.36481333
5 0.90719235
5 0.88239056
10 0.41751912
5 0.684865

14 0.7000488
14 0.7870651
11 0.3766288
10 0.5836354
14 0.79419094
14 0.32992348
4 0.31434953
14 0.35632545
11 0.33221814
14 0.43804705
11 0.36200023
11 0.40560916
11 0.30977273
11 0.4035945
11 0.36302635
14 0.4387731
11 0.37596038
5 0.53395075
14 0.36464798
4 0.635624
11 0.41877285
11 0.34395862
11 0.64623743
11 0.917802
11 0.67433125
11 0.8427403
10 0.5659864
11 0.8010315
10 0.46817768
11 0.807072
1 0.48416412
11 0.4664306
1 0.47980294
11 0.7530488
11 0.88291854
11 0.5456396
10 0.4817022
11 0.7473821
11 0.74680275
11 0.7777062
11 0.5322712
14 0.44987077
14 0.78464174
14 0.63805515
14 0.7279641
11 0.32606375
9 0.44580516
14 0.6327716
5 0.988306
5 0.70908093
5 0.4868659
11 0.82191736
5 0.5685338
5 0.83419216
5 0.971704
5 0.7315076
5 0.64077723
10 0.5885019
10 0.5244756
10 0.63982034
10 0.59759474
10 0.58036304
10 0.59593964
10 0.35992432
10 0.63668174
11 0.3726871
10 0.4457135
10 0.8060843
10 0.42019928
5 0.34957567
5 0.4765021
5 0.60762435
5 0.79788184
5 0.8333011
5 0.98905164
5 0.8181

14 0.5875488
14 0.51640296
14 0.41067973
14 0.5724501
14 0.505615
14 0.35923752
14 0.57689583
14 0.7831582
5 0.4178878
14 0.63338405
14 0.60051554
5 0.39805955
14 0.65686435
14 0.61622757
5 0.40668383
14 0.45264012
14 0.48623434
14 0.65014243
14 0.4722324
11 0.4070865
11 0.75691265
11 0.6182659
11 0.6740012
11 0.28747308
14 0.6718803
4 0.3517826
11 0.5261336
11 0.62399733
5 0.9986313
5 0.99740416
5 0.99798894
5 0.9653337
7 0.6207204
11 0.3686889
7 0.75602674
7 0.47990957
7 0.7999384
7 0.7808932
7 0.80981344
7 0.4850131
7 0.76873535
7 0.5980666
7 0.5800077
7 0.8487334
7 0.71100116
7 0.66080487
11 0.4811461
11 0.8000774
11 0.6783442
11 0.5604407
11 0.84019554
11 0.71369314
11 0.5110859
11 0.8781796
11 0.588615
5 0.35926494
11 0.50844955
11 0.56520706
11 0.4669728
5 0.50405926
11 0.4939616
5 0.74843925
5 0.79526883
5 0.5203663
5 0.67840666
5 0.5170762
5 0.7930263
5 0.81526446
5 0.86518544
5 0.7402837
5 0.8217445
5 0.8552326
5 0.8426954
5 0.93887126
5 0.9423557
5 0.89200586
5 0.85926014
5 

11 0.90058386
11 0.92740476
11 0.7353064
11 0.81311625
11 0.9182052
11 0.89506376
11 0.93533415
11 0.86082065
11 0.8805969
11 0.8201262
11 0.9519822
5 0.67564243
5 0.5804111
5 0.65832067
11 0.9648516
11 0.69957334
11 0.9641369
5 0.6117064
11 0.78494275
5 0.58990556
5 0.61794984
11 0.8008546
5 0.6146882
5 0.6759165
5 0.6265307
5 0.68782955
5 0.58193755
5 0.5800797
5 0.6468929
5 0.6694478
5 0.557024
11 0.543858
5 0.5004283
5 0.55990064
5 0.5715381
5 0.5215953
5 0.5553471
11 0.64536524
5 0.55427957
5 0.6376358
5 0.5904584
5 0.56465566
5 0.58326465
5 0.5642939
5 0.61996835
5 0.63999164
5 0.637198
5 0.5851585
5 0.5992368
5 0.7052953
5 0.66652334
5 0.6242756
5 0.7063024
5 0.66040665
5 0.6548403
5 0.7001402
5 0.7459767
5 0.7325688
5 0.7381104
5 0.7576554
5 0.6636687
5 0.80493975
5 0.85014474
5 0.7174624
5 0.9500453
5 0.967834
5 0.92235863
5 0.95227545
5 0.9350469
5 0.85062486
5 0.9092004
5 0.89399177
5 0.7485662
5 0.7103079
5 0.7263931
5 0.57480645
5 0.6325286
5 0.674265
5 0.72314143
5 0.6614

11 0.8440079
11 0.8498065
11 0.8597026
11 0.86434054
11 0.81678647
11 0.78547114
11 0.7656772
11 0.75901115
11 0.62235993
11 0.8424954
11 0.87633955
11 0.7659108
11 0.85822606
11 0.94098485
11 0.85069937
1 0.46177006
11 0.79717374
1 0.46584567
8 0.4624478
2 0.40072638
11 0.36344662
11 0.3304802
11 0.3629
8 0.39383602
1 0.6897182
1 0.8894243
1 0.88116425
1 0.8413739
1 0.55836695
1 0.88603365
1 0.82357126
1 0.8335225
1 0.79796696
1 0.7113573
1 0.7819373
1 0.9446922
1 0.7610894
1 0.64319044
1 0.7671233
1 0.7995812
1 0.7896112
1 0.47241974
1 0.5891107
8 0.5096001
1 0.47283652
10 0.4925594
2 0.36142182
10 0.48335397
10 0.31562093
2 0.51062495
8 0.80017084
8 0.41418114
1 0.6965182
1 0.8939803
1 0.8721196
1 0.7211754
1 0.5426633
1 0.6733478
1 0.7737509
1 0.5812191
5 0.81238055
5 0.63506323
5 0.82398456
11 0.40206182
11 0.50227356
11 0.56649315
11 0.80091286
14 0.53750104
14 0.37914187
14 0.6491777
14 0.44288188
14 0.43165514
5 0.43934917
5 0.45533043
5 0.6656285
5 0.6531891
5 0.662619
14 0.68